
<img style="float: right;" src="data/logo.png" width= "200" height = "200">

# Landscape evolution modelling




_____________

* Press 'shift + enter' or 'shift + command' to run a cell, or press 'Cell' menu on the top bar
* Click 'b' to make a new cell, or press 'Insert' menu on the top bar 
* To clear the whole notebook and its outputs press 'Kernel' then 'Restart and Clear Output'



In [ ]:
x = 'This is a cell, run me now' # assigns a string to a variable
x

______

### Run the  cells below to install the packages we will use:
*You only need to run these cells once, unless you restart the notebook*

In [ ]:
# !pip install badlands
# !pip install badlands-companion

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import badlands_companion.toolGeo as simple
import badlands_companion.toolTec as tec
import badlands_companion.toolSea as tools

from badlands.model import Model as badlandsModel
from scripts import catchmentErosion as eroCatch
from scripts import pyb

%config InlineBackend.figure_format = 'svg' 

______
### 1.  Initial Surface

Let's make our initial surface based on a cosine wave. 

We can change 3 parameters:

+ the amplitude of the wave (m)
+ the period of the wave (m)
+ the base of the wave (m)

In [ ]:
## YOU CAN CHANGE THESE ##

amplitude = 800 # proxy for steepness
period = 60000 # period of the surface (i.e. th)
base = -500 # bottom of the slope (m)

################

wave = simple.toolGeo(extentX=[0., 50000.], extentY=[0., 50000.], dx = 500.)
wave.Z = wave.buildWave(A = amplitude, P = period, base = base, xcenter = 0)
wave.viewGrid(width = 600, height = 600, zmin = -1000, zmax = 1000, zData = wave.Z, title= 'Export Wave Grid')

wave.buildGrid(elevation=wave.Z, nameCSV='data/xyz') # save your surface for use later in the simulation

### 2. Erodibility

Next we define how erodible our surface is

In [ ]:
## YOU CAN CHANGE THIS ##

erodibility = 3e-6

################

pyb.ero('data/xyz.csv', erodibility)


### 3. Uplift

Next we define how the surface will uplift through time with 3 different parameters:

+ the uplift rate (in m/yr),
+ the width of the uplift 'block', from xmin to xmax (in m)

In [ ]:
## YOU CHANGE THESE ##

uplift_rate = 0.0001
x_min = 0
x_max = 5000

################

stpTec = tec.toolTec(extentX=[0., 50000.], extentY=[0., 50000.], dx = 500.)
stpTec.disp = stpTec.stepTec(A = uplift_rate*100000, base = 0, edge1 = x_min, edge2 = x_max, axis='X')
stpTec.dispView(width=600, height=600, dispmin=-500, dispmax=500, dispData=stpTec.disp, title='Export Step Map')
stpTec.dispGrid(disp=stpTec.disp, nameCSV='data/uplift') # save your uplift for use later in the simulation


### 4. Sea-level

Next we define how sea-level will change through time through time:

+ the magnitiude of the sea-level change (m)
+ the period of sea-level change

In [ ]:
## YOU CAN CHANGE THIS ##

sea_fall = -30

################
    
pyb.sea_plot(sea_fall)

### 5. Run model

Now let's bring all that together and run the model

In [ ]:
## RUN THIS CELL ##

################

time = 1000000.

model = badlandsModel()
model.load_xml('input.xml')
model.run_to_time(time)

### 6. Time slices

Now we can look at some of our model results:

In [ ]:
## Set file title ##

title = 'sea_30'

crange = [-200, 200]

#################
    
pyb.time_slice(pyb.new('outputs'), 'data/sea.csv', crange, title)

### 7. Dip-sections

We could also look at dip sections to see how the surface changed through time:

In [ ]:
## Set file title ##

title = 'sea_30'

#################

# %matplotlib nbagg    

pyb.section(pyb.new('outputs'), title)
